In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
pylab.rcParams[ 'figure.figsize' ] = 10 , 8
from sklearn import preprocessing

In [222]:
Hotel_Reviews=pd.read_csv("Hotel_Reviews.csv",usecols=['Hotel_Name','Positive_Review','Negative_Review'])
TEMP=Hotel_Reviews.loc[Hotel_Reviews['Hotel_Name']=='Hotel Arena']
TEMP.head()

,Hotel_Name,Negative_Review,Positive_Review
0,Hotel Arena,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...
1,Hotel Arena,No Negative,No real complaints the hotel was great great ...
2,Hotel Arena,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...
3,Hotel Arena,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...
4,Hotel Arena,You When I booked with your company on line y...,Amazing location and building Romantic setting


In [2]:
import langid
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deniz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [210]:
corpusP= []
corpusN= []
uniqueUsefulTokens= set()

for i in TEMP.index:
       reviewP=""
       reviewN="" 
       if ("No Negative" in TEMP['Negative_Review'][i]) and "No Positive" in TEMP['Positive_Review'][i]:
            continue
       elif "No Negative" in TEMP['Negative_Review'][i]:
            reviewP = TEMP['Positive_Review'][i]
       else:
            reviewN = TEMP['Negative_Review'][i]
       
       if (langid.classify(reviewP)[0]!='en' and langid.classify(reviewN)[0]!='en'):
            continue
       reviewP= re.sub('[^a-zA-Z]', ' ', reviewP)
       reviewN= re.sub('[^a-zA-Z]', ' ', reviewN)

       reviewP=reviewP.lower().split()
       reviewN=reviewN.lower().split()

       ps=PorterStemmer()
       reviewP  = [ps.stem(word) for word in reviewP if not word in set(stopwords.words('english'))]
       reviewN  = [ps.stem(word) for word in reviewN if not word in set(stopwords.words('english'))]
       for word in reviewP:
            uniqueUsefulTokens.add(word)
       reviewP = ' '.join(reviewP)
    
       for word in reviewN:
            uniqueUsefulTokens.add(word)
       reviewN = ' '.join(reviewN)
    
       if reviewP!="":
            corpusP.append(reviewP)
        
    
       if reviewN!="":
            corpusN.append(reviewN)
            
    

In [211]:
len(corpusP)

91

In [212]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    ifdDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word] +=1
            
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [213]:
from functools import reduce

In [214]:
bowA = []
for line in corpusP:
    line= re.sub('[^a-zA-Z]', ' ', line)
    bowA.append(line.split(" "))

bowAnew = reduce(lambda x,y: x+y,bowA)
len(bowA)

91

In [215]:
wordSet=set(bowAnew)
wordSet

{'absolut',
 'access',
 'accommod',
 'ad',
 'ador',
 'advic',
 'advis',
 'age',
 'air',
 'airi',
 'allow',
 'also',
 'although',
 'alway',
 'amaz',
 'amen',
 'american',
 'amsterdam',
 'anyon',
 'appreci',
 'architectur',
 'area',
 'arena',
 'arenaaa',
 'around',
 'arriv',
 'art',
 'asid',
 'ask',
 'atmospher',
 'attent',
 'attic',
 'attract',
 'away',
 'awesom',
 'back',
 'background',
 'bad',
 'bar',
 'basic',
 'bath',
 'bathroom',
 'beauti',
 'beautiful',
 'bed',
 'bedroom',
 'besid',
 'best',
 'better',
 'big',
 'birthday',
 'bit',
 'bonu',
 'book',
 'bottl',
 'boufett',
 'boutiqu',
 'boyfriend',
 'breakfast',
 'bright',
 'brilliant',
 'broken',
 'buffet',
 'build',
 'built',
 'busi',
 'bussl',
 'cafe',
 'came',
 'cannot',
 'card',
 'care',
 'ceil',
 'center',
 'centr',
 'central',
 'ch',
 'chain',
 'chair',
 'champagn',
 'chandeli',
 'charm',
 'cheaper',
 'check',
 'checkout',
 'cheer',
 'chocol',
 'church',
 'citi',
 'classi',
 'clean',
 'cleanli',
 'clearli',
 'close',
 'closet'

In [237]:
wordDictAarr = np.array([])
for listbowA in bowA:
    wordDictA = dict.fromkeys(wordSet,0)
    for word in listbowA:
        wordDictA[word]+=1
    wordDictAarr = np.append(wordDictAarr, wordDictA)


In [231]:
i = 0
tfBowAarr = np.array([])
for listbowA in bowA:
    tfBowA = computeTF(wordDictAarr[i], listbowA)
    i +=1
    tfBowAarr = np.append(tfBowAarr, tfBowA)
tfBowAarr[0]


0.9999999999999991

In [233]:
idfs = computeIDF(wordDictAarr)
idfsData = pd.DataFrame.from_dict(idfs, orient='index', dtype=float)
idfsData[0]=1/idfsData[0]

PosRev = pd.DataFrame(idfsData[0].sort_values(ascending=False))
PosRev.iloc[0:10, :]

,0
staff,1.465807
room,1.376141
hotel,1.254253
great,0.704316
locat,0.681971
love,0.660011
help,0.660011
nice,0.638398
bed,0.575284
friendli,0.575284


In [232]:
tfidfBowA = np.array([])
for tfBowA in tfBowAarr:
    tfidf = computeTFIDF(tfBowA, idfs)
    tfidf = {x:y for x,y in tfidf.items() if y!=0}
    tfidfBowA = np.append(tfidfBowA, tfidf)


#sum(tfidfBowA[1].values())
tfidfBowA[1]

{'appreci': 0.13164525261920362,
 'breakfast': 0.08384199878748325,
 'care': 0.13164525261920362,
 'comfort': 0.06710034996742459,
 'consid': 0.15554687953506377,
 'doubl': 0.11766369716719793,
 'glass': 0.15554687953506377,
 'great': 0.09791841745920926,
 'greeneri': 0.15554687953506377,
 'hotel': 0.05498534067672706,
 'larg': 0.15554687953506377,
 'locat': 0.050563347199773345,
 'lush': 0.13164525261920362,
 'onto': 0.15554687953506377,
 'pane': 0.15554687953506377,
 'park': 0.07614739356975186,
 'plan': 0.15554687953506377,
 'price': 0.13164525261920362,
 'renov': 0.10774362570334342,
 'return': 0.10004902048561205,
 'room': 0.025057581813744446,
 'select': 0.15554687953506377,
 'spaciou': 0.10004902048561205,
 'spectacular': 0.15554687953506377,
 'structur': 0.13164525261920362,
 'uniqu': 0.15554687953506377,
 'window': 0.11766369716719793}

In [118]:
idfs = computeIDF([wordDictA])


In [119]:
tfidfBowA = computeTFIDF(tfBowA, idfs)